In [2]:
import numpy as np
from scipy.stats import norm

In [8]:
def simulate_gbm(s_0,mu,sigma,n_sims,T,N,randomseed =3):
    np.random.seed(randomseed)
    
    dt = T/N
    dW = np.random.normal(scale = np.sqrt(dt),size = (n_sims,N))
    W = np.cumsum(dW, axis = 1)
    
    time_step = np.linspace(dt,T,N)
    time_steps = np.broadcast_to(time_step,(n_sims,N))
    S_t = (s_0 * np.exp((mu - 0.5 * sigma ** 2)*time_steps +sigma *W))
    S_t = np.insert(S_t, 0, s_0, axis=1)
    
    return  S_t

Step 1: set parameters

In [9]:
S_0 = 100
K = 100
r = 0.05
sigma = 0.50
T = 1 
N = 252 
dt = T / N 
N_SIMS = 1_000_000 
discount_factor = np.exp(-r * T)

Black scholes to price a call option

In [45]:
def  black_scholes_analytical(S_0, K, T, r, sigma, type='call'):
    d1 = (np.log(S_0 / K)+(r+(0.5*sigma**2)*T))/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if type == "call":
        N_d1 = norm.cdf(d1, 0, 1)
        N_d2 = norm.cdf(d2, 0, 1)
        price = N_d1 * S_0 - N_d2 * K * np.exp(-r*T)
    elif type =="put":
        N_d1 = norm.cdf(-d1, 0, 1)
        N_d2 = norm.cdf(-d2, 0, 1)
        price = K * np.exp(-r * T) * N_d2 - S_0 * N_d1
    else:
        raise ValueError("unknow type")
    return(price)

In [46]:
black_scholes_analytical(S_0=S_0, K=K, T=T, r=r, sigma=sigma,type="call")

21.79260421286685

In [47]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, n_sims=N_SIMS, T=T, N=N)

In [48]:
gbm_option_price = (discount_factor * np.mean(np.maximum(0, gbm_sims[:, -1] - K)))
gbm_option_price

21.734629237239563

In [49]:
def simulate_gbm_option_european(s_0, K, T, r, sigma, n_sims, type='call',random_seed =3):
    np.random.seed(random_seed)
    S_T = (S_0 * np.exp((r - 0.5 * sigma ** 2)*T + sigma * np.sqrt(T) * np.random.normal(0, 1, size=n_sims)))
 
    if type == "call":    
        payoff = np.maximum(0,S_T - K)
    elif type =="put":
        payoff = np.maximum(0,K - S_T)
    else:
         raise ValueError("unknow type")
            
    optionvalue = np.exp(-r * T) * np.mean(payoff)
    
    return  optionvalue

In [50]:
simulate_gbm_option_european(S_0, K, T, r, sigma, N_SIMS, type="put")

16.868228241471794

In [56]:
def  black_scholes_analytical_with_greeks(S_0, K, T, r, sigma, type='call'):
    d1 = (np.log(S_0 / K)+(r+(0.5*sigma**2)*T))/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if type == "call":
        N_d1 = norm.cdf(d1, 0, 1)
        N_d2 = norm.cdf(d2, 0, 1)
        price = N_d1 * S_0 - N_d2 * K * np.exp(-r*T)
        Delta = N_d1
        Theta = -S_0 * norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*N_d2
        Rho = K*T*np.exp(-r*T)*N_d2
    elif type =="put":
        N_d1 = norm.cdf(-d1, 0, 1)
        N_d2 = norm.cdf(-d2, 0, 1)
        price = K * np.exp(-r * T) * N_d2 - S_0 * N_d1
        Delta = - N_d1
        Theta = -S_0*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*N_d2
        Rho = -K*T*np.exp(-r*T)*N_d2
    else:
        raise ValueError("unknow type")
    
    Gamma = norm.pdf(d1, 0, 1)/(S_0*sigma*np.sqrt(T)) 
    Vega = S_0*norm.pdf(d1, 0, 1)*np.sqrt(T)
    
    print("Option Price: ", [round(price,3)])
    print("       Delta: ", [round(Delta,3)])
    print("       Gamma: ", [round(Gamma,3)])
    print("       Vega : ", [round(Vega,3)])
    print("       Theta: ", [round(Theta,3)])
    print("       Rho  : ", [round(Rho,3)])
    
    return()  

In [57]:
black_scholes_analytical_with_greeks(S_0=S_0, K=K, T=T, r=r, sigma=sigma,type="call")

Option Price:  [21.793]
       Delta:  [0.637]
       Gamma:  [0.008]
       Vega :  [37.524]
       Theta:  [-11.476]
       Rho  :  [41.89]


()